## Решение 2 задачи от команды 20

In [2]:
import numpy as np
import matplotlib as plt
import cv2
from google.colab import files

uploaded = files.upload()

Saving 00cf8c15-5b7f-4a37-8f2d-441081838382.png to 00cf8c15-5b7f-4a37-8f2d-441081838382.png
Saving 00ea7024-ed52-4389-9605-84c980e57140.png to 00ea7024-ed52-4389-9605-84c980e57140.png
Saving 0a0a423e-a44f-43e3-b034-2a8d8ba8d2ce.png to 0a0a423e-a44f-43e3-b034-2a8d8ba8d2ce.png
Saving 0a6f13e7-1de1-4422-85fd-33c54a126ac5(1).png to 0a6f13e7-1de1-4422-85fd-33c54a126ac5(1).png
Saving 0a6f13e7-1de1-4422-85fd-33c54a126ac5.png to 0a6f13e7-1de1-4422-85fd-33c54a126ac5.png
Saving 0a638003-651f-4ee9-8f7e-32eb9c5ea585.png to 0a638003-651f-4ee9-8f7e-32eb9c5ea585.png
Saving 0ab2f67e-27dc-4922-a554-24ebf96e3948.png to 0ab2f67e-27dc-4922-a554-24ebf96e3948.png
Saving 0ada0810-16be-4bcf-a34c-bc306facb4f3.png to 0ada0810-16be-4bcf-a34c-bc306facb4f3.png
Saving 0bc3d0eb-05fc-41bd-bb09-724be6cd0e16.png to 0bc3d0eb-05fc-41bd-bb09-724be6cd0e16.png
Saving 0c123e60-88a6-4fed-b6a0-73580e7c1b6c.png to 0c123e60-88a6-4fed-b6a0-73580e7c1b6c.png
Saving 0c659d22-e5f1-4fe1-9a4a-99dfc217d2d0.png to 0c659d22-e5f1-4fe1-9a4a

### Загружаем данные для тренировки

In [45]:
def train_data_read(name):
  with open(name, 'r') as csv:
    train_f = csv.read().split('\n')[1::]
  train_0 = []
  for item in train_f:
    train_0.append(item.split(';')[1::])
  train_0.pop()
  return train_0

#labeled_ions_team_name_1.csv в той же директории
train_0 = train_data_read('labeled_ions_team_name_1.csv')

### Выделяем признаки

In [46]:
import os

def data_proc(name, folder = ""):
  path = os.path.join(folder, name)
  img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)#Считываем изображение в оттенках серого
  x4 = int(len(img)/4)

  data_x= []

  #Выделяем из изображения 4 отдельных изображения ионов и формируем для них признаки
  for i in range(4):
    img_temp = np.array(img[i*x4:(i+1)*x4])

    #Максимальное и минимальное значения пикселей изображения
    x_max = max(max(x) for x in img_temp)
    x_min = min(min(x) for x in img_temp)

    n = len(img_temp)*len(img_temp[0])
    x_ = sum(img_temp)/n

    #Среднеквадратичное отклонение
    D = np.sqrt((sum(sum((x-x_)**2) for x in img_temp))/n)

    data_x.append([x_max, x_min, D])

  return(data_x)

#нормализуем признаки
def norm(data_x):
  X_max = max(max(x) for x in data_x)
  X_min = min(min(x) for x in data_x)
  X = []
  for row in data_x:
    X.append([(x-X_min)/(X_max-X_min) for x in row])
  return X

X = []
Y = []

for item in train_0:
  folder = ""#Входные данные в той же директории, но можно изменить
  temp_data = data_proc(item[0], folder)
  for i in range(4):
    X.append(temp_data[i])
    Y.append([int(item[i+1])])

X= np.array(norm(X))#Тренировочный набор данных
Y= np.array(Y)#Ответы

print(X, Y)

[[0.18431373 0.08627451 0.11634325]
 [0.83529412 0.00392157 0.12418409]
 [0.8627451  0.00392157 0.12585908]
 [0.95294118 0.         0.13055818]
 [0.83529412 0.         0.12427671]
 [0.17254902 0.08235294 0.11632671]
 [0.7254902  0.         0.12367137]
 [0.87843137 0.00392157 0.12688652]
 [0.18431373 0.0745098  0.11594838]
 [0.88627451 0.         0.12399575]
 [0.85490196 0.00392157 0.12535266]
 [0.96078431 0.         0.12799158]
 [0.85490196 0.         0.12319038]
 [0.36470588 0.07843137 0.11792543]
 [0.95294118 0.00784314 0.12579468]
 [0.85098039 0.         0.12569026]
 [0.97647059 0.07843137 0.12450598]
 [0.87843137 0.07843137 0.1234496 ]
 [0.18039216 0.07843137 0.11762822]
 [0.96078431 0.00392157 0.12821219]
 [0.96862745 0.         0.12500492]
 [0.98431373 0.         0.12775838]
 [1.         0.01568627 0.12814321]
 [0.81568627 0.00392157 0.12551723]
 [0.97647059 0.03137255 0.12669182]
 [0.97647059 0.00784314 0.12551062]
 [0.91372549 0.00784314 0.12610759]
 [0.93333333 0.         0.12

### Строим матрицу QUBO

In [47]:
X_T = X.T
Y_T = Y.T
A = X_T.dot(X)
B = X_T.dot(Y)
C = (Y_T).dot(Y)
#print(A, B, C)
QUBO = np.array(A)
for i in range(len(B)):
  QUBO[i][i] += (-2)*B[i]
print(QUBO)

[[-36.01774702   0.44207612   4.29366236]
 [  0.44207612  -1.03051134   0.11775032]
 [  4.29366236   0.11775032  -8.8640541 ]]


### Проверяем решение

Получили результат w = [1, 0, 1], а energy+С = 1,7055236  
   
id: Classification  
  type: QUBO  
  objective:   
    target: -200   
    to: min  
  solution:  
    variables:   
      b_0: 1.0  
      b_1: 0.0  
      b_2: 1.0  
    energy: -36.2944764  
  params_best:  
    num_runs: 10  
    num_steps: 1000  
    alpha: 0.3  
    dt: 4  

In [48]:
w = np.array([1, 0, 1])

def pred(name, folder = ""):
  pred_ = []
  temp_data = np.array(norm(data_proc(name, folder)))
  for i in range(4):
    pred_.append(w.dot((np.array(temp_data[i])).T))
  return pred_

def ion_out(name, folder = "", flag_pr = True):
  ver = pred(name)
  if(flag_pr):
    print(ver)
  ion_str =""
  for i in range(len(ver)):
    if(ver[i]>= 0.35):
      ion_str += '1'
    else:
      ion_str += '0'
  return ion_str

test_0 = train_data_read('labeled_ions_team_name_2.csv')#Размечена в ручную
N_tr_te = len(train_0)*len(test_0)*4
list_pred_tr = np.array([0, 0, 0, 0])#TP, TN, FP, FN
list_pred_te = np.array([0, 0, 0, 0])

# Проверим на тренировочной выборке
for item in train_0:
  str = ''
  pred_str = ion_out(item[0])
  for i in range(4):
    str += item[1+i]
    if (item[1+i] == pred_str[i] and pred_str[i]=='1'):
      list_pred_tr[0] += 1
    elif (item[1+i] == pred_str[i] and pred_str[i]=='0'):
      list_pred_tr[1] += 1
    elif (item[1+i] != pred_str[i] and pred_str[i]=='1'):
      list_pred_tr[2] += 1
    elif (item[1+i] != pred_str[i] and pred_str[i]=='0'):
      list_pred_tr[3] += 1

  print(pred_str, str)

print('\nТест')
# И на тестовой выборке
for item in test_0:
  str = ''
  pred_str = ion_out(item[0])
  for i in range(4):
    str += item[1+i]
    if (item[1+i] == pred_str[i] and pred_str[i]=='1'):
      list_pred_te[0] += 1
    elif (item[1+i] == pred_str[i] and pred_str[i]=='0'):
      list_pred_te[1] += 1
    elif (item[1+i] != pred_str[i] and pred_str[i]=='1'):
      list_pred_te[2] += 1
    elif (item[1+i] != pred_str[i] and pred_str[i]=='0'):
      list_pred_te[3] += 1

  print(pred_str, str)

[0.3155042295875813, 1.0068598443695373, 1.0374241391659984, 1.137005501370962]
0111 0111
[1.0923685789126347, 0.3288540659615442, 0.9666794660010166, 1.1444467094126776]
1011 1011
[0.31251770504737986, 1.0515057777988681, 1.0202650183473656, 1.1332157271851928]
0111 0111
[1.0263931958957635, 0.5064649542454497, 1.1320067624184984, 1.0249013022491376]
1111 1111
[1.123988001649767, 1.0220953535152326, 0.298367722467306, 1.1116697959478938]
1101 1101
[1.0936323672106953, 1.1120721062784726, 1.1281432112452463, 0.9412035058332249]
1111 1111
[1.1297446380792542, 1.1285349674457121, 1.0648893021886927, 1.0842061323521779]
1111 1111
[1.0500445386265271, 0.29981061763605155, 1.1308844656416375, 1.0458507682032534]
1011 1011
[1.0282559044483242, 1.0443784818490431, 1.1297172872580374, 1.036778883950969]
1111 1111
[1.1207469873709686, 1.0770266264005839, 0.4753206668629615, 1.134020187719973]
1111 1111
[1.1274142245745347, 1.0236920039366517, 0.3710675011216015, 0.34171717259161644]
1110 1110



In [49]:
print(list_pred_tr, list_pred_te)
list_pred = list(map(lambda x, y: x + y, list_pred_tr, list_pred_te))
print(list_pred)
precision = list_pred[0]/(list_pred[0]+list_pred[2])
recall = list_pred[0]/(list_pred[0]+list_pred[3])
print("precision ", precision)
print("recall ", recall)
print("F1 ", 2*precision*recall/(precision+recall))

[38  6  0  0] [17  6  1  0]
[55, 12, 1, 0]
precision  0.9821428571428571
recall  1.0
F1  0.9909909909909909


### Точное решение

In [50]:
w = ((np.linalg.inv(A)).dot(B)).T[0]
print(w, ((w.T).dot(QUBO)).dot(w) + C)


list_pred_tr_w = np.array([0, 0, 0, 0])#TP, TN, FP, FN
list_pred_te_w = np.array([0, 0, 0, 0])

# Проверим на тренировочной выборке
for item in train_0:
  str = ''
  pred_str = ion_out(item[0])
  for i in range(4):
    str += item[1+i]
    if (item[1+i] == pred_str[i] and pred_str[i]=='1'):
      list_pred_tr_w[0] += 1
    elif (item[1+i] == pred_str[i] and pred_str[i]=='0'):
      list_pred_tr_w[1] += 1
    elif (item[1+i] != pred_str[i] and pred_str[i]=='1'):
      list_pred_tr_w[2] += 1
    elif (item[1+i] != pred_str[i] and pred_str[i]=='0'):
      list_pred_tr_w[3] += 1

  print(pred_str, str)

print('\nТест')
# И на тестовой выборке
for item in test_0:
  str = ''
  pred_str = ion_out(item[0])
  for i in range(4):
    str += item[1+i]
    if (item[1+i] == pred_str[i] and pred_str[i]=='1'):
      list_pred_te_w[0] += 1
    elif (item[1+i] == pred_str[i] and pred_str[i]=='0'):
      list_pred_te_w[1] += 1
    elif (item[1+i] != pred_str[i] and pred_str[i]=='1'):
      list_pred_te_w[2] += 1
    elif (item[1+i] != pred_str[i] and pred_str[i]=='0'):
      list_pred_te_w[3] += 1

  print(pred_str, str)


[ 1.12657855  1.3172007  -0.3223909 ] [[-12.32911592]]
[0.29779039506226673, 0.9509024573361883, 0.9827886674316731, 1.0824092197196107]
0111 0111
[1.0256450077096868, 0.3020870053197791, 0.8850448531089953, 1.0858906022073653]
1011 1011
[0.2793629611031935, 0.9976044934258108, 0.9657392712970113, 1.083631008431547]
0111 0111
[0.9689986897091493, 0.4996757497230151, 1.0948619256595313, 0.9635168254145666]
1111 1111
[1.18752051259492, 1.0743043067173084, 0.2676413739438936, 1.0672067456503063]
1101 1101
[1.050934458488449, 1.0677185869102688, 1.1059283131079711, 0.8836345373751847]
1111 1111
[1.1270697570365733, 1.0957199680440857, 1.023132280878944, 1.0354205573732642]
1111 1111
[0.9998807090865433, 0.27100498595625716, 1.084382585939564, 1.0001743589634875]
1011 1011
[1.0003219353725898, 0.9972074635462495, 1.0847588736295413, 0.9829968739659433]
1111 1111
[1.0813530549397847, 1.0424430714927344, 0.4654951840686544, 1.08337165767543]
1111 1111
[1.0855013600816765, 0.9728749917307056, 

In [51]:
print(list_pred_tr_w, list_pred_te_w)
list_pred_w = list(map(lambda x, y: x + y, list_pred_tr_w, list_pred_te_w))
print(list_pred_w)
precision_w = list_pred_w[0]/(list_pred_w[0]+list_pred_w[2])
recall_w = list_pred_w[0]/(list_pred_w[0]+list_pred_w[3])
print("precision ", precision_w)
print("recall ", recall_w)
print("F1 ", 2*precision_w*recall_w/(precision_w+recall_w))

[38  6  0  0] [17  7  0  0]
[55, 13, 0, 0]
precision  1.0
recall  1.0
F1  1.0


### Сформируем конечную таблицу

In [52]:
import csv
w = np.array([1, 0, 1])

# Getting the current work directory (cwd)
thisdir = os.getcwd()

Png_list = [x for x in os.listdir() if x.endswith(".png")]

with open("Out/labeled_ions_20.csv", 'w') as csv_out:
  writer = csv.writer(csv_out)
  writer.writerow(["file number", "file name", "qubit 1 state", "qubit 2 state", "qubit 3 state", "qubit 4 state"])
  for i, item in enumerate(Png_list):
    str_l = [i, item]
    ion_str_out = ion_out(item, "", False)
    for i in range(4):
      str_l.append(ion_str_out[i])
    writer.writerow(str_l)